# Demonstração de OFDM usando a Transformada Discreta de Fourier (DFT)

**Disciplina:** Processamento Digital de Sinais  
**Instituição:** Universidade Federal de Campina Grande (UFCG)  

Este notebook mostra, passo a passo, como a Transformada Discreta de Fourier (DFT) é utilizada na modulação OFDM (Orthogonal Frequency Division Multiplexing). A ideia é conectar os conceitos de DFT/IDFT vistos em Processamento Digital de Sinais com uma aplicação real em sistemas de comunicação.


## OFDM: visão conceitual

No OFDM, a banda disponível é dividida em várias **subportadoras ortogonais**. Cada subportadora carrega um símbolo complexo (por exemplo, QPSK ou QAM). Se tivermos $N$ subportadoras, podemos agrupar $N$ símbolos complexos em um vetor:

$$\mathbf{X} = [X[0], X[1], \dots, X[N-1]].$$

O transmissor gera o símbolo OFDM no tempo aplicando a IDFT:

$$x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] e^{j 2\pi kn/N}.$$

Na implementação digital, utilizamos a **IFFT** para calcular $x[n]$ de forma eficiente.
No receptor, aplicamos a **FFT** em $x[n]$ para recuperar os símbolos $X[k]$.

A ortogonalidade entre as subportadoras é garantida pela própria estrutura exponencial da DFT, com espaçamento de frequência igual a $1/N$ (em unidades normalizadas).


## Lembrando

A **Transformada Discreta de Fourier (DFT)** de uma sequência finita $x[n]$, de comprimento $N$, é definida por:

$$X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2\pi kn/N}, \quad k = 0,1,\dots,N-1.$$

A transformada inversa (IDFT) é dada por:

$$x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] e^{j 2\pi kn/N}, \quad n = 0,1,\dots,N-1.$$

No contexto de OFDM:
- $X[k]$ representa os símbolos complexos transmitidos em cada **subportadora**;
- $x[n]$ é o **símbolo OFDM** no domínio do tempo, gerado pela IDFT (na prática, via IFFT);
- No receptor, aplicamos a DFT (via FFT) para recuperar $X[k]$ a partir de $x[n]$.


In [ ]:
# Imports básicos
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)  # Para reprodutibilidade


## Configuração de um sistema OFDM simples

Vamos considerar um sistema OFDM bastante simplificado:

- Número de subportadoras $N$ (tamanho da IFFT/FFT);
- Modulação QAM simples (por exemplo, 4-QAM / QPSK);
- Canal sem ruído (AWGN pode ser introduzido como extensão);
- Sem prefixo cíclico (pode ser incluído como extensão em aula).

O foco é relacionar diretamente a IFFT/FFT com a geração/recuperação dos símbolos OFDM.


## Mapeamento de bits para subportadoras no OFDM

Nesta seção detalhamos como uma sequência de bits \(x_1, x_2, x_3, \dots\) é mapeada em símbolos complexos e, em seguida, posicionada nas subportadoras do símbolo OFDM.

### 1. Sequência de bits e agrupamento
Considere uma sequência de bits:
$$x_1, x_2, x_3, x_4, \dots$$

Se a modulação utilizada for **QPSK (4-QAM)**, cada símbolo transporta **2 bits**. Assim, agrupamos:

$$(x_1, x_2),\; (x_3, x_4),\; (x_5, x_6),\; \dots$$

De forma geral:
- Para QPSK: grupos de 2 bits;
- Para 16-QAM: grupos de 4 bits;
- Para 64-QAM: grupos de 6 bits, e assim por diante.

### 2. Mapeamento dos grupos de bits em símbolos complexos
Cada grupo de bits é convertido para um símbolo complexo de acordo com a constelação escolhida. Por exemplo, para QPSK (mapeamento Gray típico):

| Bits | Símbolo complexo |
|------|-------------------|
| 00   | \(+1 + j\)        |
| 01   | \(-1 + j\)        |
| 11   | \(-1 - j\)        |
| 10   | \(+1 - j\)        |

Após normalização (por exemplo, dividindo por $\sqrt{2}$ para energia média unitária), obtemos uma sequência de símbolos complexos:

$$s_0, s_1, s_2, \dots, s_{N-1}$$

### 3. Associação dos símbolos às subportadoras
Em OFDM, cada símbolo complexo $s_k$ modula uma subportadora de índice $k$. Construímos então um vetor no domínio da frequência:

$$\mathbf{X} = [X[0], X[1], \dots, X[N-1]]$$

Fazemos o mapeamento direto:

- $$X[0] \leftarrow s_0$$
- $$X[1] \leftarrow s_1$$
- $$\dots$$
- $$X[N-1] \leftarrow s_{N-1}$$

Ou seja, o vetor de subportadoras $X[k]$ é formado a partir dos símbolos gerados pelos grupos de bits. Na prática, algumas posições podem ser reservadas para pilotos ou banda de guarda, mas aqui assumimos todas as subportadoras úteis ocupadas para simplificar.

### 4. Geração do símbolo OFDM (ligação com a IDFT)
Uma vez definido o vetor $X[k]$, o símbolo OFDM no domínio do tempo é obtido pela IDFT:
$$x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] e^{j 2\pi kn/N}, \quad n = 0,\dots,N-1.$$

Na implementação digital, utilizamos a **IFFT** para calcular \(x[n]\) de forma eficiente. Portanto:

- Os **bits** definem os **símbolos complexos** (QPSK/QAM);
- Os símbolos complexos são colocados nas **subportadoras** (elementos de \(X[k]\));
- A IFFT gera o **símbolo OFDM** no tempo como soma de subportadoras ortogonais.


In [ ]:
# Parâmetros do sistema OFDM
N = 64  # número de subportadoras / tamanho da IFFT
M = 4   # 4-QAM (QPSK)

print(f'Número de subportadoras: {N}')
print(f'Modulação: {M}-QAM (QPSK)')

### Mapeamento de bits em símbolos 4-QAM (QPSK)

Cada símbolo 4-QAM carrega 2 bits. Vamos gerar uma sequência aleatória de bits, agrupar em pares e mapear para símbolos complexos com constelação normalizada.


In [ ]:
# Geração de bits e mapeamento 4-QAM (QPSK)
num_bits = N * 2  # 2 bits por subportadora
bits = np.random.randint(0, 2, num_bits)

def bits_to_qpsk(b):
    """Mapeia pares de bits para símbolos QPSK (4-QAM) com energia média unitária."""
    b = b.reshape(-1, 2)
    # Mapeamento Gray simples: 00->(1+1j), 01->(-1+1j), 11->(-1-1j), 10->(1-1j)
    mapping = {
        (0, 0): 1 + 1j,
        (0, 1): -1 + 1j,
        (1, 1): -1 - 1j,
        (1, 0): 1 - 1j,
    }
    symbols = np.array([mapping[tuple(bb)] for bb in b])
    # Normalização para energia média unitária
    symbols /= np.sqrt(2)
    return symbols

X = bits_to_qpsk(bits)
print('Primeiros 5 símbolos QPSK:', X[:5])

### Geração do símbolo OFDM via IFFT

O vetor de símbolos QPSK $X[k]$ será interpretado como as amostras no domínio da frequência para a IFFT. Aplicando a IFFT, obtemos o símbolo OFDM no domínio do tempo:

$$x[n] = \text{IFFT}\{X[k]\}.$$


In [ ]:
# Geração do símbolo OFDM no tempo
x_time = np.fft.ifft(X, n=N)

print('Formato de x_time:', x_time.shape)

### Visualização do símbolo OFDM no domínio do tempo

A seguir, mostramos as partes real e imaginária do símbolo OFDM gerado.


In [ ]:
# Sinal OFDM no tempo (partes real e imaginária)
n = np.arange(N)

plt.figure(figsize=(10, 4))
plt.stem(n, np.real(x_time), use_line_collection=True)
plt.xlabel('n (amostras)')
plt.ylabel('Parte real de x[n]')
plt.title('Símbolo OFDM no domínio do tempo - Parte real')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 4))
plt.stem(n, np.imag(x_time), use_line_collection=True)
plt.xlabel('n (amostras)')
plt.ylabel('Parte imaginária de x[n]')
plt.title('Símbolo OFDM no domínio do tempo - Parte imaginária')
plt.grid(True)
plt.show()

### Visualização dos símbolos no domínio da frequência

Os pontos $X[k]$ correspondem aos símbolos modulando cada subportadora. Podemos visualizar a constelação QPSK e também o módulo de $X[k]$.


In [ ]:
# Constelação QPSK e módulo dos símbolos em frequência
plt.figure(figsize=(5, 5))
plt.scatter(np.real(X), np.imag(X))
plt.xlabel('Parte real')
plt.ylabel('Parte imaginária')
plt.title('Constelação QPSK dos símbolos em subportadoras (X[k])')
plt.grid(True)
plt.axis('equal')
plt.show()

plt.figure(figsize=(10, 4))
plt.stem(np.arange(N), np.abs(X), use_line_collection=True)
plt.xlabel('Índice de subportadora k')
plt.ylabel('|X[k]|')
plt.title('Módulo dos símbolos em frequência (subportadoras)')
plt.grid(True)
plt.show()

## Recepção: aplicação da FFT

No receptor, aplicamos a FFT ao símbolo OFDM recebido no tempo. Como estamos supondo um canal ideal (sem ruído, sem multipercurso), esperamos recuperar exatamente os mesmos símbolos $X[k]$ (a menos de erros numéricos extremamente pequenos).


In [ ]:
# Receptor OFDM: aplicação da FFT
X_rec = np.fft.fft(x_time, n=N)

erro = np.max(np.abs(X - X_rec))
print('Erro máximo entre X e X_rec:', erro)

### Demapeamento para bits

A partir de $X_{\text{rec}}[k]$, fazemos uma decisão por proximidade na constelação QPSK para recuperar os bits. Com canal ideal, devemos obter exatamente a mesma sequência de bits transmitida.


In [ ]:
# Demapeamento QPSK para bits
def qpsk_to_bits(symbols):
    """Demapeia símbolos QPSK (4-QAM) para bits (decisão por quadrante)."""
    bits_rec = []
    for s in symbols:
        r = np.real(s)
        i = np.imag(s)
        if r >= 0 and i >= 0:
            bits_rec.extend([0, 0])
        elif r < 0 and i >= 0:
            bits_rec.extend([0, 1])
        elif r < 0 and i < 0:
            bits_rec.extend([1, 1])
        else:  # r >= 0 and i < 0
            bits_rec.extend([1, 0])
    return np.array(bits_rec)

bits_rec = qpsk_to_bits(X_rec)
num_erros = np.sum(bits != bits_rec)
print('Número de erros de bit (canal ideal):', num_erros)

## Discussão e conexões com a teoria de DFT

Os resultados obtidos mostram que:

1. A **IFFT** gera um sinal no domínio do tempo que é a soma de subportadoras ortogonais, cada uma modulada por um símbolo QPSK. A ortogonalidade decorre diretamente das exponenciais complexas da DFT.
2. A **FFT** no receptor recupera os símbolos originais com erro praticamente nulo, desde que o canal seja ideal. Em canais reais, precisa-se de técnicas adicionais (equalização, prefixo cíclico, etc.).
3. A DFT/IDFT fornece uma **implementação eficiente** da modulação multiportadora OFDM, substituindo bancos explícitos de osciladores senoidais por operações FFT/IFFT.

Esse exemplo pode ser estendido em sala de aula para incluir:
- Prefixo cíclico e efeito de canais com multipercurso;
- Adição de ruído AWGN e cálculo de taxa de erro de bits (BER) em função da relação $E_b/N_0$;
- Modulações de ordem maior (16-QAM, 64-QAM);
- Alocação de subportadoras piloto para estimação de canal.


## Exercícios

1. **Inserir ruído AWGN**: adicione um termo de ruído complexo ao vetor `x_time` antes da FFT no receptor. Meça a taxa de erro de bits (BER) para diferentes variâncias de ruído.
2. **Modulação 16-QAM**: modifique as funções de mapeamento/demapeamento para suportar 16-QAM e compare a BER com a de 4-QAM para o mesmo nível de ruído.
4. **Visualização da ortogonalidade**: selecione duas subportadoras específicas, gere somente essas duas, e calcule numericamente o produto interno entre elas ao longo de um símbolo OFDM. Verifique que elas são ortogonais (produto interno aproximadamente zero).
